In [1]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager
import numpy as np
from iotbx.map_model_manager import map_model_manager as MapModelManager

In [2]:
import os
import pickle
from multiprocessing import Pool
from collections import Counter

In [56]:
errors = []
error_log_path = "data/parsing_errors.txt"

maps_and_models_path = "/net/cci/share/cryoem/maps_and_models/"
entries = [entry for entry in os.listdir(maps_and_models_path) if os.path.isdir(maps_and_models_path+entry)]

entries_group_args = []

for entry in entries:
    entry_path = maps_and_models_path+entry
    filenames = os.listdir(entry_path)
    extensions = [filename.split(".")[-1] for filename in filenames]
    extension_dict = Counter(extensions)
    if extension_dict["pkl"] == 1:
        pkl_path = entry_path+"/"+[filename for filename in filenames if filename.split(".")[-1] == "pkl"][0]
        with open(pkl_path,"rb") as fh:
            group_args = pickle.load(fh)
        group_args.add(key="entry",value=entry)
        group_args.add(key="entry_pdb",value=entry[:4])
        group_args.add(key="entry_emdb",value=entry[5:])
        entries_group_args.append(group_args)

    else:
        errors.append("FAILED: "+entry_path+": .pkl occurrences != 1")

with open(error_log_path,"w") as fh:
    for error in errors:
        fh.write(error+"\n")
        

In [ ]:
def worker(group_args):

    try:
        dm = DataManager()
        dm.process_model_file(group_args.model_file)
        group_args.add(key="composition",value=dm.get_model().composition())
    except:
        group_args.add(key="composition",value="None")
    return group_args

In [48]:
p = Pool(21)
results = p.map(worker,entries_group_args)

In [49]:
with open("data/entry_composition.pkl","wb") as fh:
    pickle.dump(results,fh)

In [58]:
entry_composition_path = "data/entry_composition.pkl" #list of group_args objects
with open(entry_composition_path,"rb") as fh:
    entry_composition_list = pickle.load(fh)

In [59]:
entries_no_composition = []
entries_yes_composition = []
ligand_list = []
for entry in entry_composition_list:
    if entry.composition == "None":
        entries_no_composition.append(entry)
    else:
        entries_yes_composition.append(entry)

In [61]:
with open("data/entry_composition.pkl","wb") as fh:
    pickle.dump(entries_yes_composition,fh)